# Importamos librerias

In [1]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [2]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Subclases

In [7]:
query = '''
SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    LAA.ORIN IN ('1000050699', '1000051491', '1000157000', '1000333633', '1000056771', '1000026465')
    AND SUB.SUB_NAME NOT ILIKE '%NO%ESPECIFICADO%'
'''

cursor.execute(query)
subclases = cursor.fetch_pandas_all()
subclases.head(2)

,SUB_NAME,SUBCLASE
0,BONDIOLA,1000030001
1,LIMPIADORES LIQUIDOS MULTISUPERFICIES,8900060002


# Abandonadores

In [8]:
# antiguamente llamado Segmento 5
# Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [9]:
tuple(subclases['SUBCLASE'].astype(str))

('1000030001',
 '8900060002',
 '1200010004',
 '1100010006',
 '4000030002',
 '1700020001')

In [10]:
query = '''
WITH VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    )

SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND LSS.SCORE >= 4
        AND LSS.SUBCLASE IN {subclases_snow}
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_5 = cursor.fetch_pandas_all()

segm_5 = segm_5.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_5.shape[0])
segm_5.head(2)

144777


,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME
0,1019204,1200010004,None,98286939,MAYONESA TRADICIONAL
1,2063810,1100010006,None,93590366,SOJA


# Activos

In [11]:
# antiguamente llamado Segmento 3
# Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

In [12]:
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    ),

COMPRAS_SUBCLASES AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID || ' - ' || IM.SUBCLASE AS KEY
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    WHERE
        FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND FFM.TIEM_DIA_ID >= DATEADD(MONTH, -3, CURRENT_DATE)
    )


SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
        AND LSS.SCORE = 5
        AND LSS.SUBCLASE IN {subclases_snow}
        AND LSS.SOCI_SOCI_ID || ' - ' || LSS.SUBCLASE NOT IN (SELECT * FROM COMPRAS_SUBCLASES)
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_3 = cursor.fetch_pandas_all()

segm_3 = segm_3.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_3.shape[0])
segm_3.head(2)

157804


,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME
0,1355553,1200010004,None,95700465,MAYONESA TRADICIONAL
1,451835,1700020001,laurabique13@gmail.com,94028617,AL HUEVO


# Excels

In [15]:
for i in segm_3['SUB_NAME'].unique():
    print(i)
    df_aux = segm_3[segm_3['SUB_NAME'] == i]
    df_aux.to_excel(f'Of7 Activos - {i}.xlsx', index = False)

MAYONESA TRADICIONAL
AL HUEVO
JAMON COCIDO AL CORTE
SOJA
LIMPIADORES LIQUIDOS MULTISUPERFICIES
BONDIOLA


In [16]:
for i in segm_5['SUB_NAME'].unique():
    print(i)
    df_aux = segm_5[segm_5['SUB_NAME'] == i]
    df_aux.to_excel(f'Of7 Abandonadores - {i}.xlsx', index = False)

MAYONESA TRADICIONAL
SOJA
JAMON COCIDO AL CORTE
LIMPIADORES LIQUIDOS MULTISUPERFICIES
BONDIOLA
AL HUEVO


# Fin